In [ ]:
from main import *  # main must be imported in order to trigger the computation
from map import m

# Feature tables

# pprint(sorted(summary_sleep))
# pprint(sorted(summary_friend))
# pprint(sorted([(collapse_date_ranges(v), k) for k, v in summary_state.iteritems()]))
# pprint(sorted([(collapse_date_ranges(v), k) for k, v in summary_park.iteritems()]))
# pprint(summary_superlative_city)
# pprint(summary_superlative_nature)
# pprint(sorted(summary_animal))
# pprint(summary_got_high)
# pprint(summary_swim)
# pprint(summary_extreme_nsew)
# pprint(summary_facebook)
# pprint(summary_meal)
# pprint(summary_pie)
# pprint(summary_tiki)
# pprint(summary_wedding)
# pprint(sorted([(collapse_date_ranges(v), k) for k, v in summary_cave.iteritems()]))
# pprint(summary_elevation)
# pprint(summary_temperature)
# pprint(summary_other)

m